## Relation Extraction Model

In [1]:
!nvidia-smi

Wed Dec  1 04:19:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers==4.10.3

     |████████████████████████████████| 2.8 MB 13.4 MB/s 
     |████████████████████████████████| 3.3 MB 44.5 MB/s 
     |████████████████████████████████| 59 kB 8.9 MB/s 
     |████████████████████████████████| 895 kB 49.7 MB/s 
     |████████████████████████████████| 596 kB 42.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install pytorch_lightning==1.2.8

     |████████████████████████████████| 841 kB 12.2 MB/s 
     |████████████████████████████████| 132 kB 76.7 MB/s 
     |████████████████████████████████| 829 kB 63.8 MB/s 
     |████████████████████████████████| 329 kB 55.5 MB/s 
     |████████████████████████████████| 1.1 MB 52.5 MB/s 
     |████████████████████████████████| 192 kB 66.3 MB/s 
     |████████████████████████████████| 271 kB 62.4 MB/s 
     |████████████████████████████████| 160 kB 50.2 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=02194a468e1369f8fb2a91fcca8bce43722dc185854d9e0bc3f7ca74947689fb
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [45]:
!pip uninstall torch
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Found existing installation: torch 1.10.0
Uninstalling torch-1.10.0:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.7/dist-packages/caffe2/*
    /usr/local/lib/python3.7/dist-packages/torch-1.10.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.10.0
Looking in links: https://download.pytorch.org/whl/torch_stable.html
tcmalloc: large alloc 2041348096 bytes == 0x555ad5a8a000 @  0x7fb0536201e7 0x555ad304ef98 0x555ad3019e27 0x555ad3198115 0x555ad3132888 0x555ad301d6f2 0x555ad30fbc6e 0x555ad301d349 0x555ad310ee1d 0x555ad3090e99 0x555ad301eafa 0x555ad308cc0d 0x555ad301eafa 0x555ad308cc0d 0x555ad308bced 0x555ad301f48c 0x555ad301f698 0x555ad308dfe4 0x555ad308b9ee 0x555ad301f48c 0x555ad301f698 0x555ad308dfe4 0x555ad308b9ee 0x555ad301f48c 0x555ad301f698 0x555ad308dfe4 0x555ad308bced 0x555ad301f48c 0x555ad301f698 0x555ad308d

In [3]:
!pip uninstall torchtext
!pip install torchtext==0.10.0

Found existing installation: torchtext 0.11.0
Uninstalling torchtext-0.11.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/torchtext-0.11.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torchtext/*
Proceed (y/n)? y
  Successfully uninstalled torchtext-0.11.0
     |████████████████████████████████| 7.6 MB 5.1 MB/s 


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
### torch == 1.9.1
### transformers == 4.10.3
### pytorch_lightning == 1.2.8
import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import random
import easydict
import re
from tqdm import tqdm
from collections import Counter

from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup

from sklearn.metrics import classification_report
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

In [6]:
torch.__version__

'1.10.0+cu111'

In [7]:
pl.__version__

'1.2.8'

In [8]:
args = easydict.EasyDict({
    'seed': 42
})

In [9]:
args.batch_size = 12
args.hidden_size = 768
args.n_class = 97
args.num_workers = 4
args.epochs = 5
args.train = True
args.bert_model = 'snunlp/KR-Medium'
args.max_token_len = 512
args.train_data = '/content/drive/MyDrive/Korean_RE/data/toy_data_split/train.csv' 
args.val_data = '/content/drive/MyDrive/Korean_RE/data/toy_data_split/val.csv'
args.test_data = '/content/drive/MyDrive/Korean_RE/data/toy_data_split/test.csv'
args.relation_list = '/content/drive/MyDrive/Korean_RE/data/relation/relation_list.txt'
args.save_dir = '/content/drive/MyDrive/Korean_RE/ckpt/'
args.log_file = '/content/drive/MyDrive/Korean_RE/log/toy_result-v1.txt'
args.mode = "ALLCC"

In [10]:
args

{'batch_size': 12,
 'bert_model': 'snunlp/KR-Medium',
 'epochs': 5,
 'hidden_size': 768,
 'log_file': '/content/drive/MyDrive/Korean_RE/log/toy_result-v1.txt',
 'max_token_len': 512,
 'mode': 'ALLCC',
 'n_class': 97,
 'num_workers': 4,
 'relation_list': '/content/drive/MyDrive/Korean_RE/data/relation/relation_list.txt',
 'save_dir': '/content/drive/MyDrive/Korean_RE/ckpt/',
 'seed': 42,
 'test_data': '/content/drive/MyDrive/Korean_RE/data/toy_data_split/test.csv',
 'train': True,
 'train_data': '/content/drive/MyDrive/Korean_RE/data/toy_data_split/train.csv',
 'val_data': '/content/drive/MyDrive/Korean_RE/data/toy_data_split/val.csv'}

## 각 문장을 BERT를 통과할 수 있는 형태로 변형

함수로 작성

In [11]:
def entity_markers_added(sent: str, subj_range: list, obj_range: list) -> str:
    """ 문장과 관계를 구하고자 하는 두 개체의 인덱스 범위가 주어졌을 때 entity marker token을 추가하여 반환하는 함수.
    
    Example:
        sent = '모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.'
        subj_range = [0, 10]   # sent[subj_range[0]: subj_range[1]] => '모토로라 레이저 M'
        obj_range = [12, 21]   # sent[obj_range[0]: obj_range[1]] => '모토로라 모빌리티'
        
    Return:
        '[E1] 모토로라 레이저 M [/E1] 는  [E2] 모토로라 모빌리티 [/E2] 에서 제조/판매하는 안드로이드 스마트폰이다.'
    """
    result_sent = ''
    
    for i, char in enumerate(sent):
        if i == subj_range[0]:
            result_sent += ' [E1] '
        elif i == subj_range[1]:
            result_sent += ' [/E1] '
        if i == obj_range[0]:
            result_sent += ' [E2] '
        elif i == obj_range[1]:
            result_sent += ' [/E2] '
        result_sent += sent[i]
    if subj_range[1] == len(sent):
        result_sent += ' [/E1]'
    elif obj_range[1] == len(sent):
        result_sent += ' [/E2]'


    return result_sent.strip()
    

## Dataset 만들기

In [12]:
class KREDataset(Dataset):
    """ Dataloader for Korean Relation Extraction Dataset.
    """
    def __init__(self, data: pd.DataFrame, args):
        super().__init__()
        
        self.args = args
        self.data = data
        
        self.tokenizer = BertTokenizer.from_pretrained(args.bert_model)
        # entity markers tokens
        special_tokens_dict = {'additional_special_tokens': ['[E1]', '[/E1]', '[E2]', '[/E2]']}
        num_added_toks = self.tokenizer.add_special_tokens(special_tokens_dict)   # num_added_toks: 4
        # model.resize_token_embeddings(len(tokenizer))
        
        self.max_token_len = args.max_token_len
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx: int):
        data_row = self.data.iloc[idx]
        
        # input 문장
        sentence = data_row.sentence
        
        # subj range, obj range
        subj_range = [data_row['subj_start_pos'], data_row['subj_end_pos']]
        obj_range = [data_row['obj_start_pos'], data_row['obj_end_pos']]
        
        # input 문장 변형 - entity markers 추가: entity_markers_added 함수 이용
        converted_sent = entity_markers_added(sentence, subj_range, obj_range)
        
        labels = torch.FloatTensor(eval(data_row.label_onehot))
        
        encoding = self.tokenizer.encode_plus(
            converted_sent,
            add_special_tokens=True,
            max_length = self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )
        
        input_ids = encoding['input_ids'].flatten()
        mask = encoding['attention_mask'].flatten()
        
        return dict(sentence=converted_sent,
                   input_ids=input_ids,
                   attention_mask=mask,
                   labels=labels)

## Model

In [13]:
class KREModel(pl.LightningModule):
    """ Model for Multi-label classification for Korean Relation Extraction Dataset.
    """
    def __init__(self, args, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        
        self.args = args
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        
        self.bert = BertModel.from_pretrained(args.bert_model, return_dict=True)
        
        self.tokenizer = BertTokenizer.from_pretrained(args.bert_model)
        # entity markers tokens
        special_tokens_dict = {'additional_special_tokens': ['[E1]', '[/E1]', '[E2]', '[/E2]']}
        num_added_toks = self.tokenizer.add_special_tokens(special_tokens_dict)   # num_added_toks: 4
        
        self.bert.resize_token_embeddings(len(self.tokenizer))
        
        if self.args.mode == "ALLCC":
            self.scale = 4
        elif self.args.mode == "ENTMARK":
            self.scale = 2
            
        self.classifier = nn.Linear(self.bert.config.hidden_size * self.scale, args.n_class)
        
        self.criterion = nn.BCELoss()
        
    def forward(self, input_ids, attention_mask, labels=None):
        batch_size = input_ids.size()[0]
        
        bert_outputs = self.bert(input_ids, attention_mask=attention_mask)
        last_hidden_state = bert_outputs.last_hidden_state
        
        # 모든 entity marker의 hidden states를 concat
        if self.args.mode == "ALLCC":
            h_start_pos_tensor = (input_ids == 20000).nonzero()
            h_end_pos_tensor = (input_ids == 20001).nonzero()
            t_start_pos_tensor = (input_ids == 20002).nonzero()
            t_end_pos_tensor = (input_ids == 20003).nonzero()
            
            h_start_list = h_start_pos_tensor.tolist()
            h_end_list = h_end_pos_tensor.tolist()
            t_start_list = t_start_pos_tensor.tolist()
            t_end_list = t_end_pos_tensor.tolist()
            
            special_token_idx = []
            
            # special_token_idx example: [[1, 9, 11, 19], [3, 5, 8, 12], ..]
            for h_start, h_end, t_start, t_end in zip(h_start_list, h_end_list, t_start_list, t_end_list):
                special_token_idx.append([h_start[1], h_end[1], t_start[1], t_end[1]])
            
            # concat_state shape: [batch size, hidden size * 4]
            for i, idx_list in enumerate(special_token_idx):
                if i == 0:
                    concat_state = last_hidden_state[i, idx_list].flatten().unsqueeze(0)
                else:
                    concat_state = torch.cat([concat_state, last_hidden_state[i, idx_list].flatten().unsqueeze(0)], dim=0)
            
        elif self.args.mode == "ENTMARK":
            h_start_pos_tensor = (input_ids == 20000).nonzero()
#             h_end_pos_tensor = (input_ids == 20001).nonzero()
            t_start_pos_tensor = (input_ids == 20002).nonzero()
#             t_end_pos_tensor = (input_ids == 20003).nonzero()
            
            h_start_list = h_start_pos_tensor.tolist()
#             h_end_list = h_end_pos_tensor.tolist()
            t_start_list = t_start_pos_tensor.tolist()
#             t_end_list = t_end_pos_tensor.tolist()
            
            special_token_idx = []
        
            # special_token_idx example: [[1, 11], [3, 8], ..]
            for h_start, t_start in zip(h_start_list, t_start_list):
                special_token_idx.append([h_start[1], t_start[1]])
            
            # concat_state shape: [batch size, hidden size * 2]
            for i, idx_list in enumerate(special_token_idx):
                if i == 0:
                    concat_state = last_hidden_state[i, idx_list].flatten().unsqueeze(0)
                else:
                    concat_state = torch.cat([concat_state, last_hidden_state[i, idx_list].flatten().unsqueeze(0)], dim=0)
        
        output = self.classifier(concat_state)
        output = torch.sigmoid(output)
        
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
        return loss, output
    
    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": outputs, "labels": labels}
    
    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def training_epoch_end(self, outputs):
        labels = []
        predictions = []
        for output in outputs:
            for out_labels in output["labels"].detach().cpu():
                labels.append(out_labels)
            for out_predictions in output["predictions"].detach().cpu():
                predictions.append(out_predictions)
        labels = torch.stack(labels).int()
        predictions = torch.stack(predictions)
        
        for i in range(self.args.n_class):
            class_roc_auc = auroc(predictions[:, i], labels[:, i])
            self.logger.experiment.add_scalar(f"{str(i)}_roc_auc/Train", class_roc_auc, self.current_epoch)
            
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=2e-5)
        scheduler = get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.n_warmup_steps,
                                                    num_training_steps=self.n_training_steps)
     
        return dict(optimizer=optimizer, lr_scheduler=dict(scheduler=scheduler, interval='step'))
    

## Train Module

In [14]:
class KREDataModule(pl.LightningDataModule):
    def __init__(self, args):
        super().__init__()
        
        self.args = args
        self.batch_size = args.batch_size
        self.max_token_len = args.max_token_len
        
        self.train_df = pd.read_csv(args.train_data)
        self.val_df = pd.read_csv(args.val_data)
        self.test_df = pd.read_csv(args.test_data)
        
    def setup(self, stage=None):
        self.train_dataset = KREDataset(self.train_df, args)
        self.val_dataset = KREDataset(self.val_df, args)
        self.test_dataset = KREDataset(self.test_df, args)     
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.args.num_workers)
                         
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.args.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=self.args.num_workers)
    

In [15]:
with open(args.relation_list, 'r') as f:
    relation_list = [x.strip() for x in f.readlines()]

In [16]:
relation_list[:5]

['P17', 'P131', 'P530', 'P150', 'P47']

In [17]:
def idx2class(idx_list):
    label_out = []
    
    for idx in idx_list:
        label = relation_list[idx]
        label_out.append(label)
    return label_out if label_out else np.nan

In [18]:
idx2class([0,1,4])

['P17', 'P131', 'P47']

In [19]:
def evaluate(trained_model, test_dataset, log_filepath):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    trained_model = trained_model.to(device)
    
    predictions = []
    labels = []
    sentence_list = []
    label_list = []
    
    for i, item in enumerate(tqdm(test_dataset)):
        _, prediction = trained_model(
            item["input_ids"].unsqueeze(dim=0).to(device),
            item["attention_mask"].unsqueeze(dim=0).to(device)
        )
        
        predictions.append(prediction.flatten())
        labels.append(item["labels"].int())
        sentence_list.append(test_dataset[i]['sentence'])
        label_list.append(idx2class(np.where(test_dataset[i]['labels'] == 1)[0]))
    
    predictions = torch.stack(predictions).detach().cpu()
    labels = torch.stack(labels).detach().cpu()
    
    # [0.1, 0.2, .., 0.9]
    threshold_list = np.arange(0, 1, 0.1)[1:]
    
    print("********** Accuracy per threshold **********")
    acc_per_threshold = dict()
    for i in threshold_list:
        acc = accuracy(predictions, labels, threshold=i)
        acc_per_threshold[i] = acc
        print(f"Threshold: {i:.1f}, Accuracy: {acc:.6f}")
   
    max_acc_threshold = sorted(acc_per_threshold.items(), key=lambda x: x[1], reverse=True)[0][0]
    print(f"********** Max Threshold: {max_acc_threshold} **********")
    
    print("********** Classification Report **********")
    y_pred = predictions.numpy()
    y_true = labels.numpy()
    upper, lower = 1, 0
    y_pred = np.where(y_pred > max_acc_threshold, upper, lower)
    
    cls_report = classification_report(
        y_true,
        y_pred,
        target_names=relation_list,
        zero_division=0,
        output_dict=True
    )
    print(classification_report(
        y_true,
        y_pred,
        target_names=relation_list,
        zero_division=0
    ))
    
    # accuracy calculation
    correct = 0
    for i in range(len(predictions)):
        for j in range(len(predictions[i])):
            if predictions[i][j] < max_acc_threshold:
                predictions[i][j] = 0
            else:
                predictions[i][j] = 1
        if predictions[i].tolist() == labels[i].tolist():
            correct += 1

    acc = correct / len(predictions)
    print("***********************************************************************")
    print(f"단순 정확도(Accuracy): {acc*100:.4f}")
    print(f"조항별 정확도를 고려한 전체 정확도(Weighted Avg의 F1 Accuracy): {cls_report['weighted avg']['f1-score']*100:.4f}")
    print("***********************************************************************")
    
    ## 결과 csv 파일 저장
    result_df = pd.DataFrame(columns=['sentence', 'relation_id', 'predicted_relation_id'])
    
    result_df['sentence'] = sentence_list
    result_df['relation_id'] = label_list
    
    preds_list = []
    for i in range(len(y_pred)):
        class_pred = idx2class(np.where(y_pred[i] == 1)[0])
        preds_list.append(class_pred)
        
    result_df['predicted_relation_id'] = preds_list
    
    # result_df.to_csv('../log/results.csv', index=False)
    result_df.to_csv('/content/drive/MyDrive/Korean_RE/log/toy_results-v1.csv')
    
    with open(log_filepath, 'w') as f:
        # write args
        f.write('********** Args **********\n')
        for k in list(vars(args).keys()):
            f.write(f"{k}: {vars(args)[k]}\n")

        f.write('\n********** Accuracy per threshold **********\n')
        for k, v in acc_per_threshold.items():
            f.write(str(k)[:3] + '\t' + str(v)[7:-1] + '\n')
        f.write('\n')
        f.write('********** Max Threshold: ' + str(max_acc_threshold) + ' **********\n')
       
        f.write('\n********** Classification Report **********\n')
        f.write('라벨이름\tprecision\trecall  \tf1-score\tsupport\n')
        for k, v in cls_report.items():
            f.write(str(k) + '\t')
            f.write(f"{v['precision']:.6f}\t{v['recall']:.6f}\t{v['f1-score']:.6f}\t{v['support']}" + '\n')
        f.write('\n')
        f.write("***********************************************************************\n")
        f.write(f"단순 정확도(Accuracy): {acc*100:.4f}\n")
        f.write(f"조항별 정확도를 고려한 전체 정확도(Weighted Avg의 F1 Accuracy): {cls_report['weighted avg']['f1-score']*100:.4f}\n")
        f.write("***********************************************************************\n")


In [20]:
# if not os.path.exists('../log'):
#     os.mkdir('../log')
    
# if not os.path.exists('../ckpt'):
#     os.mkdir('../ckpt')
if not os.path.exists('/content/drive/MyDrive/Korean_RE/log'):
    os.mkdir('/content/drive/MyDrive/Korean_RE/log')

if not os.path.exists('/content/drive/MyDrive/Korean_RE/ckpt'):
    os.mkdir('/content/drive/MyDrive/Korean_RE/ckpt')
    
pl.seed_everything(args.seed)

data_module = KREDataModule(args)

Global seed set to 42


## Train

In [37]:
# train
if args.train:
    steps_per_epoch = len(data_module.train_df) // args.batch_size
    total_training_steps = steps_per_epoch * args.epochs
    warmup_steps = total_training_steps // 5
    
    model = KREModel(args, n_training_steps=total_training_steps, n_warmup_steps=warmup_steps)
    
    checkpoint_callback = ModelCheckpoint(
        dirpath=args.save_dir,
        filename="best-checkpoint",
        save_top_k=1,
        verbose=True,
        monitor="val_loss",
        mode="min"
    )
    
    logger = TensorBoardLogger("lightning_logs", name="KoreanRE")
    
    early_stopping_callback = EarlyStopping(monitor="val_loss", patience=2)
    
    trainer = pl.Trainer(
        logger=logger,
        checkpoint_callback=checkpoint_callback,
        callbacks=[early_stopping_callback],
        max_epochs=args.epochs,
        gpus=1,
        progress_bar_refresh_rate=30
    )
    
    trainer.fit(model, data_module)
    
    trainer.test()
    
    model.bert.save_pretrained(args.save_dir + '_bert')
    
# evaluate
else:
    test_dataset = KREDataset(data_module.test_df, args)
    
    trained_model = KREModel.load_from_checkpoint(os.path.join(args.save_dir, "best-checkpoint.ckpt"), args=args)
    
    trained_model.eval()
    trained_model.freeze()
    
    evaluate(trained_model, test_dataset, args.log_file)

Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Checkpoint directory /con

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: `pos_label` automatically set 1.
  warnings.warn(*args, **kwargs)


Validating: 0it [00:00, ?it/s]

...

Epoch 3, global step 23835: val_loss reached 0.00642 (best 0.00642), saving model to "/content/drive/MyDrive/Korean_RE/ckpt/best-checkpoint-v1.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Epoch 4, global step 29794: val_loss reached 0.00638 (best 0.00638), saving model to "/content/drive/MyDrive/Korean_RE/ckpt/best-checkpoint-v1.ckpt" as top 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.006425623316317797}
--------------------------------------------------------------------------------


## Evaluate

In [21]:
args.train = False

# train
if args.train:
    steps_per_epoch = len(data_module.train_df) // args.batch_size
    total_training_steps = steps_per_epoch * args.epochs
    warmup_steps = total_training_steps // 5
    
    model = KREModel(args, n_training_steps=total_training_steps, n_warmup_steps=warmup_steps)
    
    checkpoint_callback = ModelCheckpoint(
        dirpath=args.save_dir,
        filename="best-checkpoint",
        save_top_k=1,
        verbose=True,
        monitor="val_loss",
        mode="min"
    )
    
    logger = TensorBoardLogger("lightning_logs", name="KoreanRE")
    
    early_stopping_callback = EarlyStopping(monitor="val_loss", patience=2)
    
    trainer = pl.Trainer(
        logger=logger,
        checkpoint_callback=checkpoint_callback,
        callbacks=[early_stopping_callback],
        max_epochs=args.epochs,
        gpus=1,
        progress_bar_refresh_rate=30
    )
    
    trainer.fit(model, data_module)
    
    trainer.test()
    
    model.bert.save_pretrained(args.save_dir + '_bert')
    
# evaluate
else:
    test_dataset = KREDataset(data_module.test_df, args)
    
    trained_model = KREModel.load_from_checkpoint(os.path.join(args.save_dir, "best-checkpoint-v1.ckpt"), args=args)
    
    trained_model.eval()
    trained_model.freeze()
    
    evaluate(trained_model, test_dataset, args.log_file)
    

Downloading:   0%|          | 0.00/143k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/408M [00:00<?, ?B/s]

Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 22351/22351 [06:31<00:00, 57.07it/s]


********** Accuracy per threshold **********
Threshold: 0.1, Accuracy: 0.997182
Threshold: 0.2, Accuracy: 0.997739
Threshold: 0.3, Accuracy: 0.997951
Threshold: 0.4, Accuracy: 0.998062
Threshold: 0.5, Accuracy: 0.998136
Threshold: 0.6, Accuracy: 0.998151
Threshold: 0.7, Accuracy: 0.998138
Threshold: 0.8, Accuracy: 0.998084
Threshold: 0.9, Accuracy: 0.997952
********** Max Threshold: 0.6000000000000001 **********
********** Classification Report **********
              precision    recall  f1-score   support

         P17       0.95      0.96      0.96      3806
        P131       0.97      0.94      0.95      2528
        P530       0.96      0.96      0.96      1255
        P150       0.98      0.97      0.98      1120
         P47       0.93      0.91      0.92      1045
        P106       0.98      0.99      0.99      1028
         P27       0.90      0.90      0.90       980
        P461       0.98      0.97      0.98       646
        P279       0.87      0.85      0.86       628